In [ ]:
# Install necessary libraries:
# - `hmmlearn` for building Hidden Markov Models
# - `python-dotenv` for managing environment variables (not used in this script but useful for storing API keys securely)

!pip install hmmlearn python-dotenv


In [ ]:
# Define the API key and base URL for accessing CyboTrade's API

API_KEY = "gUnQPbcHdQpfH53PhGOjVE9z0vrEkIQiCIbkih868kGjEHaO"
BASE_URL = "https://api.datasource.cybotrade.rs"


In [ ]:
import numpy as np

# Backtesting function to evaluate trading strategy performance

def backtest(df):
    # Calculate percentage change in closing prices (returns)
    df['returns'] = df['close'].pct_change()
    # Multiply position (0 or 1) by returns to get strategy's return
    df['strategy_returns'] = df['position'] * df['returns']
    # Calculate Sharpe Ratio (risk-adjusted return)
    sharpe = np.mean(df['strategy_returns']) / np.std(df['strategy_returns']) * np.sqrt(252)
    # Calculate maximum drawdown (biggest drop from peak in cumulative returns)
    drawdown = (df['strategy_returns'].cumsum() - df['strategy_returns'].cumsum().cummax()).min()
    # Calculate how many times the position changed (i.e., frequency of trades)
    trade_freq = df['position'].diff().abs().sum()
    # Return strategy performance metrics
    return {
        "Sharpe Ratio": round(sharpe, 2),
        "Max Drawdown": round(drawdown, 2),
        "Trade Frequency": round(trade_freq, 2)
    }


In [ ]:
import requests
import pandas as pd

# Function to fetch historical BTC/USDT price data from CyboTrade API
def get_btc_data_cybo(symbol="BTCUSDT", interval="1h", limit=100):
    # Create full API endpoint with query parameters
    endpoint = f"/bybit-linear/candle?symbol={symbol}&interval={interval}&limit={limit}"
    url = BASE_URL + endpoint
    headers = {"X-API-Key": API_KEY}

    # Send GET request to CyboTrade API
    response = requests.get(url, headers=headers)

    # Check if request was successful
    if response.status_code == 200:
        json_data = response.json()
        # Validate that data exists and is in the correct format
        if "data" not in json_data or not isinstance(json_data["data"], list):
            raise Exception("❌ Unexpected data format received from API.")

        # Convert JSON data to Pandas DataFrame
        df = pd.DataFrame(json_data["data"])
        # Convert timestamp to datetime and set as index
        df["start_time"] = pd.to_datetime(df["start_time"], unit="ms")
        df.set_index("start_time", inplace=True)
        # Return only the relevant OHLCV columns
        return df[["open", "high", "low", "close", "volume"]]
    else:
        # Print error details and raise an exception
        print("Status code:", response.status_code)
        print("Raw response text:", response.text)
        raise Exception(f"❌ Failed to fetch data: {response.status_code}")


In [ ]:
# Function to add technical features to the DataFrame for HMM training
def add_features(df):
    # 10-period moving average
    df['ma_10'] = df['close'].rolling(window=10).mean()
    # 50-period moving average
    df['ma_50'] = df['close'].rolling(window=50).mean()
    # Rolling volatility (standard deviation)
    df['volatility'] = df['close'].rolling(window=10).std()
    # Momentum as difference from price 10 steps ago
    df['momentum'] = df['close'] - df['close'].shift(10)
    # Ratio of price to moving average (price deviation)
    df['price_to_ma'] = df['close'] / df['ma_10']
    # Drop rows with NaN values caused by rolling operations
    df.dropna(inplace=True)
    return df


In [ ]:
from hmmlearn.hmm import GaussianHMM

# Function to train a Hidden Markov Model on technical features
def train_hmm_model(df, n_states=3):
    # Select features for model training
    features = df[['momentum', 'volatility', 'price_to_ma']].values
    # Initialize Gaussian HMM with specified number of states
    model = GaussianHMM(n_components=n_states, covariance_type="full", n_iter=1000)
    # Fit HMM model to features
    model.fit(features)
    # Predict hidden states and add to DataFrame
    df['state'] = model.predict(features)
    return df, model


In [ ]:
# Function to generate buy signals based on HMM state predictions
def generate_signals(df):
    # Initialize signal column to 0 (no action)
    df['signal'] = 0
    # Assign signal = 1 to the most frequent state (assumed bullish)
    df.loc[df['state'] == df['state'].mode()[0], 'signal'] = 1
    # Shift signal to generate positions (entry is based on previous signal)
    df['position'] = df['signal'].shift().fillna(0)
    return df


In [ ]:
# Get historical price data
df = get_btc_data_cybo()
print(df.tail())

# If data has multi-level columns (unlikely here), flatten it
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(-1)

# Add technical indicators
df = add_features(df)
# Train HMM and classify states
df, model = train_hmm_model(df)
# Generate trading signals based on dominant state
df = generate_signals(df)
# Run backtest to evaluate the strategy
results = backtest(df)

# Print strategy metrics
print("\n📊 Strategy Metrics:")
for k, v in results.items():
    print(f"{k}: {v}")
